In [0]:
dim_atleta = 'workspace.project_data_football_silver.dim_atleta'
dim_campeonato = 'workspace.project_data_football_silver.dim_campeonato'
dim_clube = 'workspace.project_data_football_silver.dim_clube'
dim_estadio = 'workspace.project_data_football_silver.dim_estadio'
dim_posicao = 'workspace.project_data_football_silver.dim_posicao'
dim_rodada = 'workspace.project_data_football_silver.dim_rodada'
fato_partida = 'workspace.project_data_football_silver.fato_partida'
fato_pontuacao = 'workspace.project_data_football_silver.fato_pontuacao'

In [0]:
spark.table(fato_partida).createOrReplaceTempView("tmp_silver_partida")
spark.table(dim_estadio).createOrReplaceTempView("tmp_silver_estadio")

In [0]:
df_gold_kpi_estadios = spark.sql("""
    WITH base AS (
    -- Partidas + Estadio
    SELECT
        e.estadio_id,
        e.nome_estadio,
        f.partida_id, -- ID jogo
        f.gols_casa, -- Gols mandante
        f.gols_visitante -- Gols visitante
    FROM
        tmp_silver_partida f
        JOIN tmp_silver_estadio e
            ON f.estadio_id = e.estadio_id
    )
    SELECT
    estadio_id,
    nome_estadio,
    COUNT(partida_id) AS total_jogos, -- Total jogos estádio
    SUM(gols_casa + gols_visitante) AS total_gols, -- Gols totais
    ROUND(SUM(gols_casa + gols_visitante) / COUNT(partida_id), 2) AS media_gols_jogo, -- Média de gols por jogo
    SUM(
        CASE
        WHEN gols_casa > gols_visitante THEN 1
        ELSE 0
        END
    ) AS vitorias_mandante, -- Vitórias casa
    SUM(
        CASE
        WHEN gols_casa < gols_visitante THEN 1
        ELSE 0
        END
    ) AS derrotas_mandante, -- Derrotas casa
    SUM(
        CASE
        WHEN gols_casa = gols_visitante THEN 1
        ELSE 0
        END
    ) AS empates -- Empates
    FROM
    base
    GROUP BY
    estadio_id,
    nome_estadio
    ORDER BY
    total_jogos DESC;
""")

#salvar em delta e salvar tabela
df_gold_kpi_estadios.write\
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("workspace.project_data_football_gold.kpi_estadios")
